In [3]:
import pyodbc
import pandas as pd
import numpy as np

In [4]:
server = '(localdb)\MORFEO' 
database = 'Seguros_BdN' 
username = 'user_qa' 
password = 'qa2021seguros' 
cnxn = pyodbc.connect('DRIVER=SQL Server Native Client 11.0;SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = cnxn.cursor()

In [6]:
#4- Cargar la base y asignar la data a una variable cosechas
cosechas = pd.read_csv("Base_Seguros_Banco_del_Norte.csv", sep=";")

cosechas_carg = limpiar_columnas_csv(cosechas)
cosechas_carg.dtypes




COSECHA               object
anio_cosecha           int64
mes_cosecha            int64
OBLIGACION             int64
identifi_cliente       int64
CLIENTE               object
CONCESIONARIO         object
CIUDAD                object
confirmacion_zona     object
obs_asegurado         object
obs_ced_aseg          object
OCUPACION             object
GENERO                object
tipo_credito          object
fec_desembolso        object
fec_nacimiento        object
obs_fec_nacimiento    object
PLACA                 object
MODELO                 int64
FASECOLDA             object
fasecolda_ok           int64
SERIE                 object
MARCA                 object
CLASE                 object
ref_1                 object
ref_3                 object
vl_aseg                int64
SERVICIO              object
MEMO                  object
fec_fin_endoso        object
aseg_actual           object
inspeccion            object
tipo_poliza           object
PLAN                  object
dtype: object

In [7]:
def cargar_reporte_bd(data):
    """Funcion que realiza la carga del reporte limpio a la BD para posterior dectura desde PBI"""
    cursor = cnxn.cursor()
    for index, row in data.iterrows():
        cursor.execute("INSERT INTO [Seguros_BdN].[dbo].[seg_report_cosecha] values(?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)", 
                    row.COSECHA, row.anio_cosecha, row.mes_cosecha, row.OBLIGACION, row.identifi_cliente, row.CLIENTE, row.CONCESIONARIO, row.CIUDAD, row.confirmacion_zona, row.obs_asegurado,
                   row.obs_ced_aseg, row.OCUPACION, row.GENERO, row.tipo_credito, row.fec_desembolso, row.fec_nacimiento, row.obs_fec_nacimiento, row.PLACA, row.MODELO, row.FASECOLDA, row.fasecolda_ok,
                   row.SERIE, row.MARCA, row.CLASE, row.ref_1, row.ref_3, row.vl_aseg, row.SERVICIO, row.MEMO, row.fec_fin_endoso, row.aseg_actual, row.inspeccion, row.tipo_poliza, row.PLAN)
        cnxn.commit()
    cursor.close()

In [8]:
cargar_reporte_bd(cosechas_carg)

In [58]:
##DEFINICION DE FUNCIONES##


    

#Aseguradoras
def extraer_aseguradoras(data):
    """Funcion que realiza la extraccion de DataFrame final de las aseguradoras para cargar a BD"""
    aseguradoras_1 = cosechas["ASEGURADORA ACTUAL"]
    aseguradoras_1 = pd.Series.to_frame(aseguradoras_1)
    aseguradoras_1 = aseguradoras_1.drop_duplicates(aseguradoras_1.columns[aseguradoras_1.columns.isin(['ASEGURADORA ACTUAL'])], keep='first')
    aseguradoras_1["ASEGURADORA ACTUAL"] = aseguradoras_1["ASEGURADORA ACTUAL"].str.capitalize()  
    aseguradoras_2 = pd.read_sql("SELECT [aseguradora] AS 'ASEGURADORA ACTUAL' FROM [Seguros_BdN].[dbo].[seg_aseguradoras] WITH(NOLOCK)", cnxn)
    aseguradoras_fn = pd.concat([aseguradoras_1, aseguradoras_2])
    aseguradoras_fn = aseguradoras_fn.drop_duplicates(aseguradoras_fn.columns[aseguradoras_fn.columns.isin(['ASEGURADORA ACTUAL'])], keep='first')
    aseguradoras_fn = aseguradoras_fn.rename(columns={'ASEGURADORA ACTUAL':'Aseguradora'})
    return aseguradoras_fn

def cargar_aseguradoras_bd(data):
    """Funcion que realiza la carga del listado de aseguradoras en la base de datos"""
    cursor = cnxn.cursor()
    cursor.execute("truncate table [dbo].[seg_aseguradoras]")
    for index, row in data.iterrows():
        cursor.execute("INSERT INTO [dbo].[seg_aseguradoras] values(?)", row.Aseguradora)
        cnxn.commit()
    cursor.close()
    

#Cosechas
def extraer_cosechas(data):
    """Funcion que realiza la extraccion del DataFrame final de las cosechas para cargar a BD"""
    bases_1 = cosechas["COSECHA"]
    bases_1 = pd.Series.to_frame(bases_1)
    bases_1 = bases_1.drop_duplicates(bases_1.columns[bases_1.columns.isin(['COSECHA'])], keep='first')
    bases_1["COSECHA"] = bases_1["COSECHA"].str.capitalize()
    bases_2 = pd.read_sql("SELECT [cosecha] AS 'COSECHA' FROM [Seguros_BdN].[dbo].[seg_tipo_cosechas] WITH(NOLOCK)", cnxn)
    bases_fn = pd.concat([bases_1, bases_2])
    bases_fn = bases_fn.drop_duplicates(bases_fn.columns[bases_fn.columns.isin(['COSECHA'])], keep='first')
    bases_fn = bases_fn.rename(columns={'COSECHA':'Cosecha'})
    return bases_fn

def cargar_cosechas_bd(data):
    """Funcion que realiza la carga del listado de tipo de cosechas en la base de datos"""
    cursor = cnxn.cursor()
    cursor.execute("truncate table [dbo].[seg_tipo_cosechas]")
    for index, row in data.iterrows():
        cursor.execute("INSERT INTO [dbo].[seg_tipo_cosechas] values(?)", row.Cosecha)
        cnxn.commit()
    cursor.close()


In [59]:
bases_fn = extraer_cosechas(cosechas)
bases_fn

cargar_cosechas_bd(bases_fn)




In [38]:
df_1 = pd.read_sql("SELECT [id_aseguradora],[aseguradora] FROM [Seguros_BdN].[dbo].[seg_aseguradoras]", cnxn)

df_1 = df_1.drop_duplicates(df_1.columns[df_1.columns.isin(['aseguradora'])], keep='first')

In [34]:
cursor = cnxn.cursor()
cursor.execute("truncate table [dbo].[seg_aseguradoras]")
               
for index, row in df_1.iterrows():
     cursor.execute("INSERT INTO [dbo].[seg_aseguradoras] values(?)", row.aseguradora)
cnxn.commit()
cursor.close()

In [5]:
df_f = pd.read_sql("SELECT [aseguradora] as 'ASEGURADORA ACTUAL' FROM [Seguros_BdN].[dbo].[seg_aseguradoras]", cnxn)

df_f

,ASEGURADORA ACTUAL
0,Generali
1,Solidaria
2,Colpatria
3,Solidaria
4,Colpatria


In [116]:
#4- Cargar la base y asignar la data a una variable cosechas
cosechas = pd.read_csv("Base_Seguros_Banco_del_Norte.csv", sep=";")
cosechas.dtypes

COSECHA                                   object
OBLIGACION                                 int64
ID CLIENTE                                 int64
CLIENTE                                   object
CONCESIONARIO                             object
CIUDAD                                    object
CONFIRMACION ZONA                         object
OBSERVACION ASEGURADO                     object
OBSERVACION CEDULA ASEGURADO              object
OCUPACION                                 object
GENERO                                    object
TIPO DE CREDITO                           object
FECHA DESEMBOLSO                           int64
F.NACIMIENTO                              object
OBSERVACION FECHA DE NACIMIENTO           object
PLACA                                     object
MODELO                                     int64
FASECOLDA                                 object
FASECOLDA OK                               int64
SERIE                                     object
MARCA               

In [96]:
cosechas["MARCA"]

0          RENAULT
1            HONDA
2           NISSAN
3          RENAULT
4        CHEVROLET
5            HONDA
6       VOLKSWAGEN
7          RENAULT
8           NISSAN
9              KIA
10          NISSAN
11             KIA
12       CHEVROLET
13       CHEVROLET
14         HYUNDAI
15       CHEVROLET
16             KIA
17          SUZUKI
18          NISSAN
19       CHEVROLET
20         RENAULT
21             KIA
22       CHEVROLET
23            AUDI
24          NISSAN
25       CHEVROLET
26           MAZDA
27       CHEVROLET
28         RENAULT
29         HYUNDAI
           ...    
3867        TOYOTA
3868     CHEVROLET
3869        NISSAN
3870        NISSAN
3871       HYUNDAI
3872     CHEVROLET
3873       CITROEN
3874       RENAULT
3875        NISSAN
3876     CHEVROLET
3877        NISSAN
3878        NISSAN
3879     CHEVROLET
3880    MITSUBISHI
3881       RENAULT
3882     CHEVROLET
3883    VOLKSWAGEN
3884     CHEVROLET
3885       HYUNDAI
3886     CHEVROLET
3887           KIA
3888       R

,COSECHA,OBLIGACION,ID CLIENTE,CLIENTE,CONCESIONARIO,CIUDAD,CONFIRMACION ZONA,OBSERVACION ASEGURADO,OBSERVACION CEDULA ASEGURADO,OCUPACION,...,REFERENCIA 3,VL ASEGURADO CONFIRMADO,SERVICIO,MEMO,FECHA FIN ENDOSO CONFIRMADA,ASEGURADORA ACTUAL,INSPECCION POR VIGENCIA Y/O SINIESTRO,CAZADOR,TIPO DE POLIZA,PLAN
19,Cosecha Historica,2700008684,17808362,CARRENO JAIMES LUIS ARTURO,DINISSAN,Barranquilla (Atl.),BARRANQUILLA,CARREÑO JAIMES LUIS ARTURO/FINANDINA,17808362/8600518946,INDEPEND/CONTRATISTA,...,MT 2400CC 4X2 AA,37400000,PARTICULAR,LLEGA POR CORREO 13/06/2016 PLACA DCS247 ...,42706,SURAMERICANA,INSPECCION POR VIGENCIA Y SINIESTRO/2014,NaN,ENDOSO EXTERNO,TRADICIONAL
20,Cosecha Historica,1300163818,66744692,PAREDES MINOTTA NIDIA,YANACONAS MOTOR,Buenabentura (Valle),BUENAVENTURA,PAREDES MINOTTA NIDIA,66744692,ADMINISTRADOR,...,MT 1400CC 4P AA AB,24100000,PARTICULAR,LLEGO FISICO 25/01/16 PLACA MXM922 ...,42806,ESTADO,INSPECCION POR SINIESTRO/2016,NaN,ENDOSO EXTERNO,TRADICIONAL
21,Cosecha Historica,2100210538,2916106,DUARTE R LUIS ANTONIO,CASA TORO AUTOMOTRIZ,BOGOTA D.C (CUND.),BOGOTA,DUARTE RAMIREZ LUIS ANTONIO/FINANDINA,2916106/8600518946,PROFESIONAL,...,MT 1600CC AA,22300000,PARTICULAR,LLEGA POR CORREO 21/07/2016 PLACA KJS978 ...,42805,ESTADO,NaN,NaN,ENDOSO EXTERNO,TRADICIONAL
22,Cosecha Historica,2100209112,41770707,CARMONA CUADROS LUZ ELENA,METROKIA,BOGOTA D.C (Cund.),BOGOTA,CARMONA CUADROS LUZ ELENA/FINANDINA,41770707/8600518946,ANALISTA,...,MT 2000CC 1AB,39000000,PARTICULAR,LLEGA POR CORREO 25/07/2016 PLACA HAQ466 ...,42795,ALLIANZ,NaN,NaN,ENDOSO EXTERNO,TRADICIONAL
23,Cosecha Historica,2300055818,6060475,ACOSTA R FABIO,AUTOPACIFICO,Cali (Valle),CALI,ACOSTA RIOS FABIO/FINANDINA,6060475/8600518946,JUBILADO / PENSIONADO,...,TP 1800CC,45200000,PARTICULAR,LLEGA POR CORREO 04/03/2016 PLACA KJT005 ...,42444,MAPFRE,INSPECCION POR VIGENCIA,NaN,ENDOSO EXTERNO,TRADICIONAL
24,Cosecha Historica,2100210636,900113696,CONSULTORIA INMOBILIARIA CORPORATIVA CI,COLWAGEN,Chia (Cund.),BOGOTA,OSORIO VELEZ SANTIAGO/FINANDINA,19267622/8600518946,INDEPENDIENTE / PROPIETARIO DE NEGOCIO,...,TP 1400CC T CT,60900000,PARTICULAR,LLEGA POR CORREO 15/03/2016 PLACA MWX009 ...,42795,ALLIANZ,NaN,NaN,ENDOSO EXTERNO,TRADICIONAL
25,Cosecha Historica,1100535948,51737453,MONCAYO V ADRIANA XIMENA,DINISSAN,BOGOTA D.C (Cund.),BOGOTA,MONCAYO V ADRIANA XIMENA,51737453,ANALISTA,...,AT 1600CC 2AB,26200000,PARTICULAR,RECIBIDO X CORRES JULIAN 30/01/2015 PL:HAR5...,42812,MAPFRE,NaN,NaN,ENDOSO EXTERNO,TRADICIONAL
26,Cosecha Historica,1600120733,71681631,RAMIREZ LONDONO GILDARDO JAVIER,ANDAR,Envigado (Ant),MEDELLIN,RAMIREZ LONDONO GILDARDO JAVIER,71681631,GERENTE,...,AT 1600CC 5P 4AB ABS CT,35200000,PARTICULAR,LLEGA POR CORREO 17/06/2016 PLACA NET865 ...,42795,ALLIANZ,NaN,NaN,ENDOSO EXTERNO,TRADICIONAL


In [41]:
def extraer_csv_aseguradoras(data):
    """Funcion que realiza la extraccion del DatFrame de aseguradoras"""
    aseguradoras_1 = cosechas["ASEGURADORA ACTUAL"]
    aseguradoras_1 = pd.Series.to_frame(aseguradoras_1)
    aseguradoras_1["ASEGURADORA ACTUAL"] = aseguradoras_1["ASEGURADORA ACTUAL"].str.capitalize()
    return aseguradoras_1

In [42]:
aseg = extraer_csv_aseguradoras(cosechas)
aseg

,ASEGURADORA ACTUAL
0,Liberty
1,Bolivar
2,Bolivar
3,Liberty
4,Mapfre
5,Generali
6,Bolivar
7,Allianz
8,Previsora
9,Suramericana


In [29]:
aseguradoras_1 = cosechas["ASEGURADORA ACTUAL"]
aseguradoras_1 = pd.Series.to_frame(aseguradoras_1)
aseguradoras_1

,ASEGURADORA ACTUAL
0,LIBERTY
1,BOLIVAR
2,BOLIVAR
3,LIBERTY
4,MAPFRE
5,GENERALI
6,BOLIVAR
7,ALLIANZ
8,PREVISORA
9,SURAMERICANA


In [34]:
#aseguradoras = pd.merge(df_f, aseguradoras_1, left_on='aseguradora', right_on='ASEGURADORA ACTUAL')
aseguradoras = pd.concat([df_f, aseguradoras_1])
aseguradoras["ASEGURADORA ACTUAL"] = aseguradoras["ASEGURADORA ACTUAL"].str.lower()
aseguradoras

,ASEGURADORA ACTUAL
0,generali
1,solidaria
2,colpatria
3,solidaria
4,colpatria
0,liberty
1,bolivar
2,bolivar
3,liberty
4,mapfre


In [121]:
#Aseguradoras
def extraer_csv_aseguradoras(data):
    """Funcion que realiza la extraccion del DatFrame de aseguradoras"""
    aseguradoras_1 = cosechas["ASEGURADORA ACTUAL"]
    aseguradoras_1 = pd.Series.to_frame(aseguradoras_1)
    aseguradoras_1 = aseguradoras_1.drop_duplicates(aseguradoras_1.columns[aseguradoras_1.columns.isin(['ASEGURADORA ACTUAL'])], keep='first')
    aseguradoras_1["ASEGURADORA ACTUAL"] = aseguradoras_1["ASEGURADORA ACTUAL"].str.capitalize()
    return aseguradoras_1
    
#Concesionarios
def extraer_csv_concesionarios(data):
    """Funcion que realiza la extraccion del DatFrame de aseguradoras"""
    concesionarios_1 = cosechas["CONCESIONARIO"]
    concesionarios_1 = pd.Series.to_frame(concesionarios_1)
    concesionarios_1 = concesionarios_1.drop_duplicates(concesionarios_1.columns[concesionarios_1.columns.isin(['CONCESIONARIO'])], keep='first')
    concesionarios_1["CONCESIONARIO"] = concesionarios_1["CONCESIONARIO"].str.capitalize()
    return concesionarios_1
    
#Clase Vehiculo
def extraer_csv_clase_veh(data):
    """Funcion que realiza la extraccion del DatFrame de aseguradoras"""
    clasevehi_1 = cosechas["CLASE"]
    clasevehi_1 = pd.Series.to_frame(clasevehi_1)
    clasevehi_1 = clasevehi_1.drop_duplicates(clasevehi_1.columns[clasevehi_1.columns.isin(['CLASE'])], keep='first')
    clasevehi_1["CLASE"] = clasevehi_1["CLASE"].str.capitalize()
    return clasevehi_1
    
#Vehiculos
def extraer_csv_vehiculos(data):
    """Funcion que realiza la extraccion del DatFrame de aseguradoras"""
    vehiculos_1 = cosechas.loc[:,["MARCA","CLASE","REFERENCIA 1","REFERENCIA 3", "VL ASEGURADO CONFIRMADO"]]
    #vehiculos_1 = pd.Series.to_frame(vehiculos_1)
    vehiculos_1 = vehiculos_1.drop_duplicates(vehiculos_1.columns[vehiculos_1.columns.isin(['MARCA','CLASE','REFERENCIA 1','REFERENCIA 3','VL ASEGURADO CONFIRMADO'])], keep='first')
    vehiculos_1["MARCA"] = vehiculos_1["MARCA"].str.capitalize()
    vehiculos_1["CLASE"] = vehiculos_1["CLASE"].str.capitalize()
    vehiculos_1["REFERENCIA 1"] = vehiculos_1["REFERENCIA 1"].str.capitalize()
    return vehiculos_1

#Servicio Vehiculo
def extraer_csv_serv_vehiculo(data):
    """Funcion que realiza la extraccion del DatFrame de aseguradoras"""
    serv_veh_1 = cosechas["SERVICIO"]
    serv_veh_1 = pd.Series.to_frame(serv_veh_1)
    serv_veh_1 = serv_veh_1.drop_duplicates(serv_veh_1.columns[serv_veh_1.columns.isin(['SERVICIO'])], keep='first')
    serv_veh_1["SERVICIO"] = serv_veh_1["SERVICIO"].str.capitalize()
    return serv_veh_1

#Ciudades
def extraer_csv_ciudades(data):
    """Funcion que realiza la extraccion del DatFrame de ciudades"""
    ciudades_1 = cosechas["CONFIRMACION ZONA"]
    ciudades_1 = pd.Series.to_frame(ciudades_1)
    ciudades_1 = ciudades_1.drop_duplicates(ciudades_1.columns[ciudades_1.columns.isin(['CONFIRMACION ZONA'])], keep='first')
    ciudades_1["CONFIRMACION ZONA"] = ciudades_1["CONFIRMACION ZONA"].str.capitalize()
    return ciudades_1

#Clientes
def extraer_csv_clientes(data):
    """Funcion que realiza la extraccion del DatFrame de clientes"""
    clientes_1 = cosechas.loc[:,["ID CLIENTE","CLIENTE"]]
    #vehiculos_1 = pd.Series.to_frame(vehiculos_1)
    clientes_1 = clientes_1.drop_duplicates(clientes_1.columns[clientes_1.columns.isin(['ID CLIENTE','CLIENTE'])], keep='first')
    clientes_1["CLIENTE"] = clientes_1["CLIENTE"].str.upper()
    return clientes_1

#Cosechas
def extraer_csv_cosechas(data):
    """Funcion que realiza la extraccion del DatFrame de ciudades"""
    bases_1 = cosechas["COSECHA"]
    bases_1 = pd.Series.to_frame(bases_1)
    bases_1 = bases_1.drop_duplicates(bases_1.columns[bases_1.columns.isin(['COSECHA'])], keep='first')
    bases_1["COSECHA"] = bases_1["COSECHA"].str.capitalize()
    return bases_1

In [122]:
bases_1 = extraer_csv_cosechas(cosechas)
bases_1

,COSECHA
0,Cosecha historica
2012,Base normal
3329,Cosecha


In [100]:
vehiculos_1 = cosechas.loc[:,["MARCA","CLASE","REFERENCIA 1","REFERENCIA 3", "VL ASEGURADO CONFIRMADO"]]
vehiculos_1

,MARCA,CLASE,REFERENCIA 1,REFERENCIA 3,VL ASEGURADO CONFIRMADO
0,RENAULT,AUTOMOVIL,SANDERO [FL],MT 1600CC 8V AA,23800000
1,HONDA,CAMIONETA PASAJ.,CRV [2],AT 2400CC,27800000
2,NISSAN,AUTOMOVIL,TIIDA SD,MT 1800CC AA,27900000
3,RENAULT,AUTOMOVIL,SANDERO [FL],MT 1600CC 8V AA,25800000
4,CHEVROLET,AUTOMOVIL,SAIL,MT 1400CC 4P AA AB,24100000
5,HONDA,AUTOMOVIL,CIVIC [9],AT 1800CC CT,54900000
6,VOLKSWAGEN,AUTOMOVIL,GOL [5],MT 1600CC 4P AA VED 2AB,26600000
7,RENAULT,AUTOMOVIL,CLIO II,MT 1200CC 16V AA DA,20100000
8,NISSAN,AUTOMOVIL,MARCH,MT 1600CC,23100000
9,KIA,CAMIONETA PASAJ.,SPORTAGE FQ [2],MT 2000CC AB,44000000
